# Learning Objectives:

### - Create SQL DBs and populate them using Python

### - Interact with SQL DBs using Python and Pandas

### - Understand Joins Using SQL

--------------------------------------

### In order to run SQL scripts from Python we are going to need a few things.
#### - SQL Database.
#### - Established connection.
#### - Familiarity with "How to SQL."

------

### The sqlite3 package

The command line utility can be useful for basic SQL tasks, but since we're using Python for the rest of code it will often be easier to access sqlite directly from within Python. We can use the Python sqlite3 package for just this purpose.

Open a connection to an SQLite database file. As before, if the file does not already exist it will automatically be created.

If you want to see your data in SQL DB form you can download a SQLite Manager.

You can download a very limited trial version here: http://www.sqlabs.com/sqlitemanager.php



In [ ]:
import sqlite3

# Establishing our Database path.  If no database exists here, this will create one.
sqlite_db = 'data/sql/test_db.sqlite'

# Establishing the SQL Connection to our Database.  
conn = sqlite3.connect(sqlite_db)

c = conn.cursor()
# Cursor objects allow you to keep track of which result set is which, 
# since it's possible to run multiple queries before you're done fetching the results of the first.
# CURSORS seem to be a tool for iterating over tables row-by-row
# This is a conceptual SQL object that is hard to give a very clear consise definition of.

The syntax to create a table is similar to the console, only now we use the execute method of the cursor object c that we just created:

In [ ]:
# We can use the 'execute' function on our Database Connection (With Cursor) to execute a SQL Query.
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')

# sqlite.connect('data/sql/test_db.sqlite').cursor().execute()

# Save (commit) the changes (Just like GitHub)
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection. In particular, the cursor's execute() method supports value substitution using the ? character, which makes adding multiple records a bit easier. Check out documentation for more details.

In [ ]:
# We can create a tuple
last_sale = (None, 4000, 5, 22, 619000)

# We can insert said tuple into the database table using character substitution.
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', last_sale)

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python None value, rather than NULL, to trigger SQLite to auto-increment the Primary Key.

There is a related cursor method executemany() which takes an array of tuples and loops through them, substituting one tuple at a time.

In [ ]:
# We can use the same logic to insert a whole list of tuples!
# Tell me thats not helpful!
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

# c.execute('DELETE FROM houses WHERE rowid > 8')

conn.commit()

Once again, use SQLite Manager to verify the database contents.

# Pandas and SQL

#### Pandas connector

While databases provide many analytical capabilities, often it's useful to pull the data back into Python for it's more flexible programming operations. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales to present a report or dashboard, this operation is likely not changing and operating on a large dataset. This can run very efficiently in a database rather than by connecting to it with Python.

However, if we want to investigate login or sales data further and ask more interactive questions, then Python would be more practical.

## Pandas Functions for SQL

#### read_sql_table(table_name, con[, schema, ...])
    - Read SQL database table into a DataFrame.
#### read_sql_query(sql, con[, index_col, ...])
    - Read SQL query into a DataFrame.
#### read_sql(sql, con[, index_col, ...])
    - Read SQL query or database table into a DataFrame.
    - a convenience wrapper around read_sql_table() and read_sql_query()
    - will delegate to specific function depending on the provided input
#### DataFrame.to_sql(name, con[, flavor, ...])
    - Write records stored in a DataFrame to a SQL database.

#  Create a SQL DB and tables using Pandas DFs and CSVs

First we will need to read our CSV files into Python before we can use Python to convert it to a SQL style dataframe

In [ ]:
import pandas as pd
from pandas.io import sql

cars = pd.read_csv('data/csv/car-names.csv', encoding = 'utf-8')
# If you don't specify the type encoding as 'utf-8' you're going to have a bad time when you try to convert to SQL


In [ ]:
# Checking what our dataframe looks like
cars.head(3)

In [ ]:
# Checking for nulls in our data
cars.info()

In [ ]:
# Import Sqlite3 Library
import sqlite3

# Establishing the Connection to our Database.  If no database exists here, this will create one.
connection = sqlite3.connect('data/sql/Cars.db.sqlite')

# Keep in mind the directory your notebook is open in is the base directory for all of our SQL actions from here.

#### If you check that directory now you should see an 'Cars.db' sql file.

In [ ]:
# Converts a DataFrame into a SQL database
cars.to_sql(name = 'car_names', con = connection, if_exists = 'replace', index = False)

# name = name of the database useful if you have multiple tables in a SQL database
# con = the connection path to where the data should be placed
# if_exists = condition to pass if the database already exists.

#### Using the below will allow you to acess a database store in memory(RAM) as opposed to in Storage, if you wanted a temporary SQL database

``` python
conn = sqlite3.connect(':memory:')
```

In [ ]:
# Creating a Table for Order Breakdowns
makers = pd.read_csv('data/csv/car-makers.csv', encoding = 'utf-8')

connection = sqlite3.connect('data/sql/Cars.db.sqlite')

makers.to_sql(name = 'car_makers', con = connection, if_exists = 'replace', index = False)


In [ ]:
# Creating a Table for the Sales Targets
data = pd.read_csv('data/csv/cars-data.csv',encoding = 'utf-8')

connection = sqlite3.connect('data/sql/Cars.db.sqlite')

data.to_sql(name = 'car_data', con = connection, if_exists = 'replace', index = False)


In [ ]:
# When reading in data to a SQL DB from pandas, how to specify a Key in the table.

# Connect to SQL DB and read SQL tables into Pandas DFs
#### (Also, return to SQL DataFrames modified tables)

In [ ]:
#The SQL Sub-library from Pandas will allow us to run SQL queries within python.
from pandas.io import sql
# We already imported sqlite3, but it will also be needed for reading in SQL 
import sqlite3

# Specifying the SQL Path to the SQL Database
connection = sqlite3.connect('data/sql/Cars.db.sqlite')

# This is our SQL Query
query = 'select * from car_names'

# Using the read_sql from the Pandas SQL library and setting it equal to a DF object.
cars = sql.read_sql(query, con = connection)

cars.head()

#  Run SQL Queries using Pandas

In [ ]:
sql.read_sql('select * from car_names', connection).head()

#sql = Our SQL query as a string
#con = is the connection path to our SQL Database

In [ ]:
# Figure out how to read all the table names in a connection.

# 4. Review Joins

## Normalized and Denormalized Databases

There are several ways to organize data in a relational database. Two common definitions for data setups are: normalized and denormalized.

__Normalized__ structures have a single table per entity, and use many foreign keys or link tables to connect the entities.

__Denormalized__ tables have fewer tables and may (for example) place all of the tweets and the information on users in one table.

Each style has advantages and disadvantages. Denormalized tables duplicate a lot of information. For example, in a combined tweets/users table, we may store the address of each user. Now instead of storing this once per user, we are storing this once per tweet!

However, this makes the data easy to access if we ever need to find the tweet along with the user's location.

Normalized tables save the storage space by separating the information. However, if we ever need to access those two pieces of information, we would need to join the two tables, which can be a fairly slow operation.

## Join Types

SQL joins are used when data is spread in different tables. A join operation allows to combine rows from two or more tables in a single new table. In order for this to be possible, a common field between the tables need to exist.

Join operations can be thought of as operations between two sets, where records with the same key are combined and records missing in one set are either discarded or included as NULL values.

_CHECK: _
- _Where have you encountered a similar functionality in Pandas?_
- _Can you make a couple of examples of how you used that Pandas function in the past?_

![join types](images/joins.gif)

Join Types:
- INNER JOIN: Returns all rows when there is at least one match in BOTH tables
- LEFT JOIN: Return all rows from the left table, and the matched rows from the right table
- RIGHT JOIN: Return all rows from the right table, and the matched rows from the left table
- FULL JOIN: Return all rows when there is a match in ONE of the tables

# SQL Joins
![sql join types](images/sql-joins.jpeg)

In [ ]:
# In the case that typing out sql.read_sql() is a little too much,
# we'll create a function shortcut.


CARS = sqlite3.connect('data/sql/Cars.db.sqlite')


def Q(query, db=CARS):
    return sql.read_sql(query, db)

In [ ]:
Q('select * from car_names limit 5')

In [ ]:
new_car = (None, 'Ferrari','The Ferrari')
CARS.execute('INSERT INTO car_names VALUES (?, ?, ?)',new_car)

In [ ]:
new_car = [None, 'Tesla', None]
CARS.execute('INSERT INTO car_names VALUES (?, ?, ?)',new_car)

In [ ]:
Q('SELECT * FROM car_names WHERE car_names."Model" = "Tesla"')

In [ ]:
Q('select * from car_makers limit 5')

In [ ]:
Q('select * from car_data limit 5')

#### Order ID is our matching feature that we can use to merge.

#### Lets Checkout all the ways we can merge these.

## Inner

The most common type of join is: SQL INNER JOIN (simple join). An SQL INNER JOIN returns all rows from multiple tables where the join condition is met. 

In our example, an INNER JOIN on "CustomerID" takes the intersection of the two datasets, excluding the rows for which CustomerID is null in EITHER of the two tables.

Essentially, only matching pairs of Order ID's from both Datasets will be taken.

In [ ]:
inner_join = Q('SELECT car_names."Make", car_data."MPG", car_data."Horsepower", car_data."Year" '
'FROM car_names '
'INNER JOIN car_data '
'ON car_names."Id"=car_data."Id"')
inner_join.head()

In [ ]:
inner_join.info()

## Left

The LEFT JOIN keyword returns all rows from the left table (table1), with the matching rows in the right table (table2). The result is NULL in the right side when there is no match.

In [ ]:
left_join = Q('SELECT car_names."Make", car_data."MPG", car_data."Horsepower", car_data."Year" '
'FROM car_names '
'LEFT JOIN car_data '
'ON car_names."Id"=car_data."Id"')
left_join.head()

In [ ]:
left_join.info()

## Right

Similarly, the RIGHT JOIN keyword returns all rows from the right table (table2), with the matching rows in the left table (table1). The result is NULL in the left side when there is no match.

## Full Outer

The FULL OUTER JOIN keyword returns all rows from the left table (table1) and from the right table (table2). The FULL OUTER JOIN keyword combines the result of both LEFT and RIGHT joins. In this case we could have NULL values on both sides.

No examples for RIGHT and FULL OUTER because, they are not supported in this relation.  

## Addtional Resources:

Kind of long winded, but a good resource as far as explaining Pandas functions from a SQL programmers perspective.
(The opposite situation of us.)

Pydata Video:
https://www.youtube.com/watch?v=1uVWjdAbgBg

Assciated GitHub Repo:
https://github.com/gjreda/pydata2014nyc/tree/master/data



Pandas Merge, Join and Concatenate
http://pandas.pydata.org/pandas-docs/stable/merging.html